# Projeto 3 - Ciência dos Dados

#  Projeto UFC

Nome: Victor Vergara Arcoverde de Albuquerque Cavalcanti

Nome: Edgard Neto

Nome: Gabriel Yamashita

Nome: Henrique Mualem Marti



 ___
## Preparando o ambiente no jupyter:

### Imports:

In [55]:
import math
import os.path
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import random

import statsmodels.api as sm

import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

### Trabalhando com os Excels:

In [2]:
data = pd.read_excel("data.xlsx")
preproc = pd.read_excel("preprocessed_data.xlsx")
fighter = pd.read_excel("raw_fighter_details.xlsx")
total = pd.read_excel("raw_total_fight_data.xlsx")

data.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,...,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,...,0,2,0,5,Southpaw,165.10,167.64,125.0,32.0,31.0


In [3]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

In [4]:
data.Winner.value_counts()

Red     3470
Blue    1591
Draw      83
Name: Winner, dtype: int64

In [5]:
data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data_heavy['title_bout'] = data_heavy['title_bout'].map(bool_to_number)
data_heavy['Winner'] = data_heavy['Winner'].map(string_to_number)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


#### Blue = 0
#### Red = 1

In [6]:
data_heavy.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019,"Chicago, Illinois, USA",0,0,Heavyweight,3,0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,Jerin Valel,2019,"Ottawa, Ontario, Canada",1,0,Heavyweight,3,0,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0


In [18]:
data_heavy_util = data_heavy.drop(['Referee','date','location'], axis=1)
#dados que não se relacionam com os lutadores ou seus resultado

In [19]:
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,1,0,Heavyweight,3,0,0,0,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.0,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.0,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.0,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0


In [54]:
data_heavy_util.dropna(inplace=True)
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.00,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
1,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.00,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
2,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.00,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
3,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.00,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0
4,Alistair Overeem,Aleksei Oleinik,1,0,Heavyweight,5,0,2,0,4.25,...,6,0,0,9,Orthodox,193.04,203.20,265.0,41.0,38.0


In [26]:
categoricas = [
    'R_fighter', 
    'B_fighter', 
    'weight_class', 
    'R_Stance', 
    'B_Stance', 
]

data_cat = data_heavy_util[categoricas].astype('category')
data_num = data_heavy_util.drop(categoricas, axis=1).astype('float')

In [27]:
X = data_num.drop('Winner', axis=1)
y = data_num['Winner']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [33]:

model = RandomForestClassifier(n_estimators=10000)

model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [34]:
for i, f in sorted(list(zip(model.feature_importances_, X_train.columns)), reverse=True):
    print(f'{f}: {i}')

R_total_rounds_fought: 0.01337106013693753
R_avg_opp_GROUND_att: 0.012757604766742463
B_avg_TOTAL_STR_landed: 0.012551156522133167
R_avg_opp_GROUND_landed: 0.012518123376598017
R_avg_opp_CLINCH_landed: 0.012309152152385124
B_avg_TOTAL_STR_att: 0.012085296929015516
R_losses: 0.012068408326694901
R_avg_opp_TOTAL_STR_landed: 0.011672316251026636
R_age: 0.011593459700009079
R_avg_BODY_att: 0.011327076876586213
B_avg_GROUND_landed: 0.01127662799905286
R_avg_HEAD_att: 0.011271161407958674
R_avg_HEAD_landed: 0.011195264004016072
R_avg_SIG_STR_att: 0.01113682197280602
B_avg_GROUND_att: 0.011082638800344538
B_Reach_cms: 0.011068228185722813
B_avg_SIG_STR_landed: 0.01104108861071061
B_avg_opp_TOTAL_STR_landed: 0.010925183294703836
R_avg_opp_CLINCH_att: 0.010713457285137304
B_avg_HEAD_att: 0.010476904913202247
R_avg_opp_SIG_STR_pct: 0.010399438080786427
B_avg_opp_BODY_landed: 0.010243270327275676
R_avg_SIG_STR_landed: 0.01020336753850784
B_avg_opp_SIG_STR_pct: 0.010179598913959823
B_avg_HEAD_land

In [35]:
X_train.columns

Index(['title_bout', 'no_of_rounds', 'B_current_lose_streak',
       'B_current_win_streak', 'B_draw', 'B_avg_BODY_att', 'B_avg_BODY_landed',
       'B_avg_CLINCH_att', 'B_avg_CLINCH_landed', 'B_avg_DISTANCE_att',
       ...
       'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Height_cms', 'R_Reach_cms',
       'R_Weight_lbs', 'B_age', 'R_age'],
      dtype='object', length=136)

In [36]:
model.feature_importances_

array([0.00043152, 0.00099733, 0.00282873, 0.00326461, 0.        ,
       0.00994718, 0.00986212, 0.00949418, 0.00972177, 0.00803171,
       0.00818039, 0.01108264, 0.01127663, 0.0104769 , 0.01006656,
       0.0058111 , 0.00744672, 0.00785877, 0.00615597, 0.00530973,
       0.00870157, 0.01104109, 0.0084788 , 0.00500721, 0.00814413,
       0.0073371 , 0.00835714, 0.0120853 , 0.01255116, 0.00385689,
       0.00483093, 0.00979006, 0.01024327, 0.00862235, 0.00771963,
       0.00920751, 0.00611429, 0.00960005, 0.00971539, 0.00945645,
       0.0077081 , 0.0051601 , 0.00961751, 0.00736837, 0.00634827,
       0.00358111, 0.00818001, 0.00701341, 0.0101796 , 0.00421932,
       0.00849181, 0.00736336, 0.00745883, 0.01000288, 0.01092518,
       0.00715959, 0.00834296, 0.00175385, 0.00050248, 0.00166883,
       0.00171315, 0.00517461, 0.00178854, 0.00068317, 0.00398794,
       0.00591689, 0.01106823, 0.00886753, 0.00354987, 0.00834007,
       0.        , 0.01132708, 0.00916246, 0.00846085, 0.00818

In [37]:
y_pred = model.predict(X_test)

In [38]:
print(accuracy_score(y_test, y_pred))

0.6515151515151515


In [39]:
y_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.])

In [40]:
y_test.value_counts(True)

1.0    0.651515
0.0    0.348485
Name: Winner, dtype: float64

In [41]:
data_num.shape

(264, 137)

In [42]:
data_heavy_util.dropna(inplace=True)

In [43]:
for c in data_heavy_util.columns:
    print(c)

R_fighter
B_fighter
Winner
title_bout
weight_class
no_of_rounds
B_current_lose_streak
B_current_win_streak
B_draw
B_avg_BODY_att
B_avg_BODY_landed
B_avg_CLINCH_att
B_avg_CLINCH_landed
B_avg_DISTANCE_att
B_avg_DISTANCE_landed
B_avg_GROUND_att
B_avg_GROUND_landed
B_avg_HEAD_att
B_avg_HEAD_landed
B_avg_KD
B_avg_LEG_att
B_avg_LEG_landed
B_avg_PASS
B_avg_REV
B_avg_SIG_STR_att
B_avg_SIG_STR_landed
B_avg_SIG_STR_pct
B_avg_SUB_ATT
B_avg_TD_att
B_avg_TD_landed
B_avg_TD_pct
B_avg_TOTAL_STR_att
B_avg_TOTAL_STR_landed
B_longest_win_streak
B_losses
B_avg_opp_BODY_att
B_avg_opp_BODY_landed
B_avg_opp_CLINCH_att
B_avg_opp_CLINCH_landed
B_avg_opp_DISTANCE_att
B_avg_opp_DISTANCE_landed
B_avg_opp_GROUND_att
B_avg_opp_GROUND_landed
B_avg_opp_HEAD_att
B_avg_opp_HEAD_landed
B_avg_opp_KD
B_avg_opp_LEG_att
B_avg_opp_LEG_landed
B_avg_opp_PASS
B_avg_opp_REV
B_avg_opp_SIG_STR_att
B_avg_opp_SIG_STR_landed
B_avg_opp_SIG_STR_pct
B_avg_opp_SUB_ATT
B_avg_opp_TD_att
B_avg_opp_TD_landed
B_avg_opp_TD_pct
B_avg_opp_TOTAL

In [44]:
data_heavy_util = data_heavy_util.apply(lambda x: pd.Series(x.dropna().values))
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.00,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
1,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.00,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
2,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.00,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
3,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.00,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0
4,Alistair Overeem,Aleksei Oleinik,1,0,Heavyweight,5,0,2,0,4.25,...,6,0,0,9,Orthodox,193.04,203.20,265.0,41.0,38.0


In [45]:
data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]
data_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)
data_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]
data_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)

In [46]:
blue_winners_correlation = data_heavy_blue_winners.corr()
blue_winners_correlation.head()

,date,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
date,1.000000,-0.161123,-0.032762,0.054294,-0.145747,NaN,0.274744,0.256441,0.154872,0.179436,...,0.231107,0.255263,-0.078268,-0.051332,0.205434,0.029779,0.010070,0.050460,0.102656,0.258130
title_bout,-0.161123,1.000000,0.491461,-0.137166,0.345926,NaN,0.000122,0.002365,0.059811,0.056089,...,0.198632,0.182796,0.064792,-0.056728,0.181773,-0.018567,0.100335,-0.185558,0.014377,-0.040564
no_of_rounds,-0.032762,0.491461,1.000000,0.197561,0.187900,NaN,0.074386,0.052264,0.007436,-0.003680,...,0.254201,0.301632,0.121706,0.012118,0.299674,0.046856,0.168529,0.049829,0.330574,0.089774
B_current_lose_streak,0.054294,-0.137166,0.197561,1.000000,-0.372420,NaN,-0.157427,-0.157421,-0.037084,-0.058432,...,-0.164409,-0.110859,0.015331,0.150660,-0.098475,0.045668,-0.029027,0.085575,0.228017,-0.107595
B_current_win_streak,-0.145747,0.345926,0.187900,-0.372420,1.000000,NaN,0.091439,0.080253,-0.048230,-0.055570,...,0.196682,0.400372,0.109060,-0.045517,0.351582,0.015903,0.152685,0.149758,0.039346,0.154148


In [47]:
red_winners_correlation = data_heavy_red_winners.corr()
red_winners_correlation.head()

,date,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
date,1.000000,-0.198749,0.318576,0.024610,0.051611,NaN,0.357148,0.312028,0.174556,0.199298,...,0.340822,0.348328,0.055707,0.067783,0.346713,0.242605,0.200622,0.264429,0.169596,0.347192
title_bout,-0.198749,1.000000,0.554955,-0.122654,0.398417,NaN,-0.074743,-0.062833,-0.041575,-0.015233,...,0.167790,0.117978,0.044928,-0.013669,0.138938,-0.025717,-0.001179,-0.171403,-0.010161,0.074430
no_of_rounds,0.318576,0.554955,1.000000,-0.051759,0.399056,NaN,0.105310,0.093346,0.052067,0.087419,...,0.403303,0.374827,0.118545,0.040138,0.411639,0.178414,0.108087,0.043754,0.096232,0.213947
B_current_lose_streak,0.024610,-0.122654,-0.051759,1.000000,-0.368059,NaN,-0.001175,-0.037899,0.015732,-0.027961,...,-0.102451,-0.023346,-0.024777,-0.036201,-0.055483,-0.000604,-0.052664,0.095878,0.095483,-0.007968
B_current_win_streak,0.051611,0.398417,0.399056,-0.368059,1.000000,NaN,0.000361,0.020183,0.007000,0.050790,...,0.355833,0.300663,0.111235,0.091870,0.347008,0.114947,0.111624,-0.024794,0.025129,0.148722


In [48]:
def preparo(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp,missing='drop')
    results = model.fit()
    return results

In [49]:
Y = data_heavy_util["Winner"]
data_heavy_sem_Winner=data_heavy_util.drop('Winner',axis=1) 
X=data_heavy_sem_Winner
np.asarray(X)

array([['Tai Tuivasa', 'Blagoy Ivanov', 0, ..., 264.0, 32.0, 26.0],
       ['Arjan Bhullar', 'Juan Adams', 0, ..., 245.0, 27.0, 32.0],
       ['Greg Hardy', 'Dmitrii Smoliakov', 0, ..., 265.0, 36.0, 30.0],
       ...,
       ['Andrei Arlovski', 'Vladimir Matyushenko', 0, ..., 240.0, 32.0,
        24.0],
       ['Josh Barnett', 'Randy Couture', 1, ..., 250.0, 38.0, 24.0],
       ['Randy Couture', 'Vitor Belfort', 0, ..., 205.0, 20.0, 34.0]],
      dtype=object)

In [50]:
result = preparo(X,Y)
result.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

___
### Montando a Regressão Logística


___
### Verificando a performance


___
### Concluindo

Após a análise é possível concluir que o classificador criado não é eficaz para a classificação, pois apresenta apenas 30% de acerto, assim não deveria ser utilizado para a classificação de twitts em níveis de relevância para o setor de marketing da "Coca-Cola". Caso fossem utilizados mais twitts para o treinamento, provavelmete o índice de acerto seria maior, pois assim teria um aumento na quantidade de palavras do sistema, principalmente para os "Muito Relevantes",que não tem muitos exemplos, assim não tendo 1 único acerto pelo classificador, e os "Neutros" que por terem não terem muita diferença dos "Inrrelevantes" e "Relevantes" 

   ___
## Aperfeiçoamento:


   ___
## Referências

[Como funciona regressão linear](https://www.saedsayad.com/logistic_regression.htm)

[Como usar a biblioteca Scikit-lear](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)